# Pack around ligand docks

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/grad_reqs
running on node: dig107


### Insert the root of the crispy_shifty repo

In [2]:
sys.path.insert(0, "/projects/crispy_shifty")

### Run MPNN on the interfaces

In [ ]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "00_pack_around_ligand"
design_list_file = "/home/pleung/projects/grad_reqs/00_pack_around_ligand/docks.list"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")
options = " ".join(
    [
        "out:level 200",
        "corrections:beta_nov16 true",
        "extra_res_fa /home/pleung/projects/grad_reqs/00_pack_around_ligand/afo.params",
    ]
)

gen_array_tasks(
    distribute_func="deployables.pack_around_ligand",
    func_root="/home/pleung/projects/grad_reqs",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="4G",
    nstruct=1,
    nstruct_per_task=50,
    options=options,
    simulation_name=simulation_name,
)

### Collect scorefiles of the MPNN designed bound states and concatenate

In [ ]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "00_pack_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [ ]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

### Dump scores_df as a CSV and then reload, for performance reasons

In [ ]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Data exploration

In [ ]:
print(len(scores_df))
print(list(scores_df.columns))

### Prototyping blocks